# 0. Importing packages

In [1]:
# Load the autoreload extension to automatically reload modules before executing code (to avoid restarting the kernel)
%load_ext autoreload 

# Enable autoreload for all modules
%autoreload 2

# Python 
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import json

# Custom Packages
from resources.github_functions import GithubScraper

## 0.1 File paths ###

In [2]:
fp_main = Path('/Volumes/SAM-SODAS-DISTRACT/Coding Distraction/github_as_a_market_device')
fp_main_output = Path(fp_main / 'output')

# 1. Scraping users from the initial company list

## 1.2 Processing the company list into:
1. Dictionary containing company categories --> company_category
2. A zip_file containing the company-string to query when scraping plus a location_filter dummy, indicating whether the location filter should apply when queriyng that company --> user_location_zip

In [3]:
# Load the company list from Excel
company_list_full = pd.read_excel(
    fp_main / 'company_list' / 'company_info_list211022.xlsx',
    usecols='A:M'
)

# Filter for companies that are part of our sample
company_list_sample = company_list_full.loc[
    company_list_full['part_of_firmaliste'] == 1
].reset_index(drop=True)

# Create a list of relevant company info
list_of_company_names = company_list_sample[
    ['søgeord', 'new_company_category', 'uden lokation']
]

# Create a dictionary mapping company name (lowercase) to category
company_category = dict(zip(
    company_list_sample['søgeord'].str.lower(),
    company_list_sample['new_company_category']
))

# Write the dictionary as JSONL
with open(fp_main_output / 'company_category_dict.jsonl', 'w', encoding='utf-8') as f:
    for company, category in company_category.items():
        json.dump({'søgeord': company, 'new_company_category': category}, f, ensure_ascii=False)
        f.write('\n')

In [4]:
# Create a zip object for name and location filter status
company_location_filter_bool_zip = zip(
    list_of_company_names['søgeord'],
    list_of_company_names['uden lokation'],
    company_list_sample['company_label_name']
)

# 2. Queriyng the company names and scraping users

## 2.1 Loading in logs in case scrape has been interrupted

In [5]:
# Collecting users already scraped
users_already_scraped = set()
companies_already_scraped = set()

user_log_file_name = 'first_tier_userinfo_user_log.jsonl'
company_log_file_name = 'first_tier_userinfo_company_log.jsonl'
user_log_path = fp_main_output / user_log_file_name
company_log_path = fp_main_output / company_log_file_name

# Ensure files exist and print message
if user_log_path.exists():
    print(f"[INFO] User log exists: {user_log_path.name}")
else:
    print(f"[INFO] User log does NOT exist. Creating new file: {user_log_path.name}")
    user_log_path.touch(exist_ok=True)

if company_log_path.exists():
    print(f"[INFO] Company log exists: {company_log_path.name}")
else:
    print(f"[INFO] Company log does NOT exist. Creating new file: {company_log_path.name}")
    company_log_path.touch(exist_ok=True)

# Read user log
with open(user_log_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            user_info = json.loads(line)
            users_already_scraped.add(user_info['user_login'])
        except (json.JSONDecodeError, KeyError) as err:
            print(f"[WARNING] Skipping malformed user line: {err}")

# Read company log
with open(company_log_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            company = json.loads(line)
            companies_already_scraped.add(company['company_name'])
        except (json.JSONDecodeError, KeyError) as err:
            print(f"[WARNING] Skipping malformed company line: {err}")

[INFO] User log exists: first_tier_userinfo_user_log.jsonl
[INFO] Company log exists: first_tier_userinfo_company_log.jsonl


## 2.2 Instantiating the GithubScraper and scrape

In [ ]:
# 1. Create instance of GithubScraper
gs = GithubScraper(
    users_already_scraped=users_already_scraped,
    companies_already_scraped=companies_already_scraped,
    repo_limit=300
)
print(f'GitHub REST API ratelimit reset time for token {gs.current_token_key} is {gs.reset_time_point}. '
      f'That will be in a little less than {gs.reset_time_in_minutes} minutes.')

# 2. Define output file name
file_name = 'first_tier_userinfo'

# 3. Loop through company queries
for search_query, skip_location_filter, company_label in tqdm(company_location_filter_bool_zip, total=len(company_list_sample)):

    # 3.1 Skip company if already scraped
    if company_label in gs.companies_already_scraped:
        print(f'[INFO] Company {company_label} already scraped. Skipping.')
        continue

    print(f'[INFO] Scraping users for company: {company_label}')
    
    # 3.2 Get users for this company
    users = gs.get_gh_users(search_query, skip_location_filter)

    # 3.3 Loop through users
    for named_user, company in users:

        # 3.3.1 Skip user if already scraped
        if named_user.login in gs.users_already_scraped:
            print(f'[INFO] User {named_user.login} already scraped. Skipping.')
            continue

        print(f'[INFO] Scraping user: {named_user.login}')
        gs.users_already_scraped.add(named_user.login)

        # 3.3.2 Get user info (may return None if repo limit exceeded or no match)
        user_row = gs.get_user_info(named_user, company_label)
        if user_row is None:
            continue  # Skip user if they don't meet scraping criteria

        # 3.3.3 Extract match data
        location_match = user_row.location
        inferred_company = user_row.inferred_company
        matched_company_strings = user_row.matched_company_strings

        # 3.3.4 Save user info and log result
        gs.save_file(user_row, file_name, remove_existing_file=True)
        gs.log_user_w_match(named_user.login, inferred_company, matched_company_strings, location_match, user_log_path)
        
        print(f'[INFO] {gs.USERS_SCRAPED} users scraped so far.')

    # 3.4 Log company after scraping all users
    gs.log_company(company_label, company_log_path)

GithubScraper initialized with 3 tokens.
First token in cycle. Initiating ACCESS_TOKEN_1.
GithubScraper initialized with 35 companies and 185 users already scraped.
GitHub REST API ratelimit reset time for token ACCESS_TOKEN_1 is 2025-05-28 14:29:41. That will be in a little less than 30 minutes.


  0%|          | 0/64 [00:00<?, ?it/s]

[INFO] Company nodes already scraped. Skipping.
[INFO] Company abtion already scraped. Skipping.
[INFO] Company heyday already scraped. Skipping.
[INFO] Company trifork already scraped. Skipping.
[INFO] Company frontit already scraped. Skipping.
[INFO] Company holion already scraped. Skipping.
[INFO] Company kruso already scraped. Skipping.
[INFO] Company pandiweb already scraped. Skipping.
[INFO] Company uptime already scraped. Skipping.
[INFO] Company charlie tango already scraped. Skipping.
[INFO] Company ffw already scraped. Skipping.
[INFO] Company mysupport already scraped. Skipping.
[INFO] Company shape already scraped. Skipping.
[INFO] Company makeable already scraped. Skipping.
[INFO] Company mustache already scraped. Skipping.
[INFO] Company house of code already scraped. Skipping.
[INFO] Company greener pastures already scraped. Skipping.
[INFO] Company axla already scraped. Skipping.
[INFO] Company snapp already scraped. Skipping.
[INFO] Company appscaptain already scraped.

Request GET /users/elsewhat failed with 403: Forbidden
Setting next backoff to 581.257467s


User match PederHP logged.
[INFO] 186 users scraped so far.
[INFO] Scraping user: michelandresaxo
User match michelandresaxo logged.
[INFO] 187 users scraped so far.
[INFO] Scraping user: jorgeta
User match jorgeta logged.
[INFO] 188 users scraped so far.
[INFO] Scraping user: phccdk
User match phccdk logged.
[INFO] 189 users scraped so far.
[INFO] Scraping user: hannefolmer


...

 81%|████████▏ | 52/64 [1:01:39<14:20, 71.67s/it]

User match pwcdk-emil logged.
[INFO] 222 users scraped so far.
Company pwc logged.
[INFO] Scraping users for company: studiesandme
[INFO] Scraping user: StudiesAndMe
[INFO] Scraping user: studiesandme-machine


 83%|████████▎ | 53/64 [1:01:42<09:25, 51.37s/it]

Company studiesandme logged.
[INFO] Scraping users for company: tv2
[INFO] Scraping user: tv2
User match tv2 logged.
[INFO] 223 users scraped so far.
[INFO] Scraping user: quartercastle
Token cycled to ACCESS_TOKEN_2.
Cycle


Request GET /users/m110 failed with 403: Forbidden
Setting next backoff to 1222.600527s
